# **Gold Layer Scripts**


### Loading ZedaFleet Silver Lakehouse Tables

In [13]:
customers = spark.read.table("ZedaFleet_Silver.customers")
customer_interactions = spark.read.table("ZedaFleet_Silver.customer_interactions")
sales = spark.read.table("ZedaFleet_Silver.sales")
rentals = spark.read.table("ZedaFleet_Silver.rentals")
maintenance = spark.read.table("ZedaFleet_Silver.maintenance")
vehicles = spark.read.table("ZedaFleet_Silver.vehicles")
contracts = spark.read.table("ZedaFleet_Silver.contracts")
telemetry = spark.read.table("ZedaFleet_Silver.telemetry")

StatementMeta(, 30be3c13-53c6-49b7-9afc-df9b95f831fb, 15, Finished, Available, Finished)

### Import Libraries

In [14]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, coalesce, lit, round as spark_round, trim

StatementMeta(, 30be3c13-53c6-49b7-9afc-df9b95f831fb, 16, Finished, Available, Finished)

### Creating and load ZedaFleet Lakehouse Gold Tables

In [18]:
# Customer 360

customer_360 = (
    customers.alias("c")
    .join(rentals.alias("r"), "customer_id", "left")
    .join(contracts.alias("co"), "customer_id", "left")
    .join(customer_interactions.alias("ci"), "customer_id", "left")
    .groupBy(
        "c.customer_id", "c.customer_name", "c.customer_segment",
        "c.country", "c.city", "c.registration_date",
        "c.loyalty_points", "c.preferred_branch", "c.marketing_consent"
    )
    .agg(
        countDistinct("r.rental_id").alias("total_rentals"),
        countDistinct("co.contract_id").alias("total_leases"),
        countDistinct("ci.interaction_id").alias("total_interactions"),
        sum("r.total_amount").alias("total_rental_revenue"),
        sum("co.monthly_payment").alias("total_lease_value")
    )
)

customer_360.write.format("delta")\
            .mode("overwrite")\
            .saveAsTable("customer_360")

StatementMeta(, 30be3c13-53c6-49b7-9afc-df9b95f831fb, 20, Finished, Available, Finished)

In [19]:
# Vehicle Lifecycle

vehicle_lifecycle = (
    vehicles.alias("v")
    .join(sales.alias("s"), "vehicle_id", "left")
    .join(rentals.alias("r"), "vehicle_id", "left")
    .join(contracts.alias("co"), "vehicle_id", "left")
    .join(maintenance.alias("m"), "vehicle_id", "left")
    .groupBy(
        "v.vehicle_id", "v.registration", "v.make", "v.model", "v.segment",
        "v.acquisition_date", "v.purchase_price", "v.current_value",
        "v.status", "v.odometer", "v.warranty_expiry",
        "s.sale_date", "s.sale_price", "s.profit_margin"
    )
    .agg(
        countDistinct("r.rental_id").alias("rental_count"),
        sum("r.total_amount").alias("rental_revenue"),
        countDistinct("co.contract_id").alias("lease_count"),
        sum("co.monthly_payment").alias("lease_revenue"),
        countDistinct("m.maintenance_id").alias("maintenance_events"),
        sum("m.cost").alias("total_maintenance_cost"),
        sum("m.downtime_days").alias("total_downtime")
    )
)

vehicle_lifecycle.write.format("delta")\
                 .mode("overwrite")\
                 .saveAsTable("vehicle_lifecycle")

StatementMeta(, 30be3c13-53c6-49b7-9afc-df9b95f831fb, 21, Finished, Available, Finished)

In [21]:
# Maintenance Summary

maintenance_summary = (
    maintenance.alias("m")
    .join(vehicles.alias("v"), "vehicle_id", "left")
    .groupBy("v.vehicle_id", "v.make", "v.model")
    .agg(
        count("m.maintenance_id").alias("maintenance_events"),
        sum("m.cost").alias("total_maintenance_cost"),
        round(avg("m.cost"), 2).alias("avg_maintenance_cost"),
        sum("m.downtime_days").alias("total_downtime"),
        max("m.service_date").alias("last_service_date")
    )
)

maintenance_summary.write.format("delta")\
                   .mode("overwrite")\
                   .saveAsTable("maintenace_summary")

StatementMeta(, 30be3c13-53c6-49b7-9afc-df9b95f831fb, 23, Finished, Available, Finished)

In [24]:
# Customer Interaction Summary

customer_interaction_summary = (
    customer_interactions.alias("ci")
    .join(customers.alias("c"), "customer_id", "left")
    .groupBy("c.customer_id", "c.customer_name")
    .agg(
        count("ci.interaction_id").alias("total_interactions"),
        max("ci.interaction_date").alias("last_interaction_date"),
        count(when(col("ci.followup_required") == "YES", 1)).alias("pending_followups")
    )
)

customer_interaction_summary.write.format("delta")\
                            .mode("overwrite")\
                            .saveAsTable("customer_interaction_summary")

StatementMeta(, 30be3c13-53c6-49b7-9afc-df9b95f831fb, 26, Finished, Available, Finished)